# GraphRAG Python package End-to-End Example

This notebook contains an end-to-end worked example using the [GraphRAG Python package](https://neo4j.com/docs/neo4j-graphrag-python/current/index.html) for Neo4j. It starts with unstructured documents (in this case pdfs), and progresses through knowledge graph construction, knowledge graph retriever design, and complete GraphRAG pipelines. 

Research papers on Lupus are used as the data source. We design a couple of different retrievers based on different knowledge graph retrieval patterns. 

For more details and explanations around each of the below steps, see the [corresponding blog post](https://neo4j.com/blog/graphrag-python-package/) which contains a full write-up, in-depth comparison of the retrieval patterns, and additional learning resources.

Setup the LLM endpoints on the GPU workstation:

    # 1. vLLM (real 4-bit)
    docker run -d --rm --gpus all -p 8001:8000 \
      -v $PWD/cache:/root/.cache/huggingface \
      vllm/vllm-openai:latest \
      --model Qwen/Qwen2-7B-Instruct-AWQ \
      --quantization awq \
      --served-model-name qwen2 \
      --dtype float16 \
      --port 8000

    # 2. Infinity (smaller warm-up)
    docker run -d --rm --gpus all -p 7997:7997 \
      -v $PWD/cache:/app/.cache \
      michaelf34/infinity:latest-trt-onnx \
      v2 --model-id BAAI/bge-large-en-v1.5 \
      --served-model-name bge \
      --batch-size 8 \
      --dtype float16 --device cuda --port 7997


Confirm that the endpoints work:

    % http POST http://10.0.1.37:8001/v1/chat/completions \
         model=qwen2 \
         messages:='[
           {"role": "system", "content": "You are a helpful assistant."},
           {"role": "user",   "content": "Hello, world!"}
         ]' \
         temperature:=0.2 \
         max_tokens:=128

    {
        "choices": [
            {
                ...
                "message": {
                    "content": "Hello! It's nice to meet you. How can I assist you today?",
                    ...
                },
                ...
            }
        ],
        ...
    }

    % http POST http://10.0.1.37:7997/embeddings \
      model=bge \
      input:='["Natural-language string …"]'

    {
        "created": 1749391848,
        "data": [
            {
                "embedding": [
                    0.02315451204776764,
                    -0.02632719837129116,
        -0.01859377510845661,
                    0.018731053918600082,
                    0.07425306737422943,
                    -0.0092587536200881,
                    ...
                    0.00983075238764286,
                    0.007191931363195181
                ],
                "index": 0,
                "object": "embedding"
            }
        ],
        "id": "infinity-247581f8-4ae2-46f5-b62b-359eb461df7f",
        "model": "bge",
        "object": "list",
        "usage": {
            "prompt_tokens": 25,
            "total_tokens": 25
        }
    }



In [1]:
%%capture
%pip install fsspec langchain-text-splitters tiktoken openai python-dotenv numpy torch neo4j-graphrag==1.7.0

In [2]:
import os
os.environ["OPENAI_API_BASE"] = "http://10.0.1.37:8001/v1"
os.environ["OPENAI_API_KEY"] = "dummy"

EMBEDDING_BASE  = "http://10.0.1.37:7997"
EMBEDDING_MODEL = "bge"

In [3]:
from dotenv import load_dotenv
import os

# load neo4j credentials (and openai api key in background).
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

#uncomment this line if you aren't using a .env file
# os.environ['OPENAI_API_KEY'] = 'copy_paste_the_openai_key_here'

## Knowledge Graph Building

The `SimpleKGPipeline` class allows you to automatically build a knowledge graph with a few key inputs, including
- a driver to connect to Neo4j,
- an LLM for entity extraction, and
- an embedding model to create vectors on text chunks for similarity search.

There are also some optional inputs, such as node labels, relationship types, and a custom prompt template, which we will use to improve the quality of the knowledge graph. For full details on this, see [the blog](https://neo4j.com/blog/graphrag-python-package/).


In [4]:
from openai import OpenAI
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

LLM_BASE_URL = "http://10.0.1.37:8001/v1"   # vLLM server
EMBEDDING_BASE_URL  = "http://10.0.1.37:7997"      # Infinity server
EMBEDDING_MODEL = "bge"

def make_clients():
    # --- Embeddings ---------------------------------------------------------
    embeddings = OpenAIEmbeddings(
        EMBEDDING_MODEL,
        base_url=EMBEDDING_BASE,   # http://10.0.1.37:7997
        api_key="local",           # dummy key; server ignores it
        timeout=30.0, 
    )

    # --- LLM ---------------------------------------------------------------
    llm = OpenAILLM(
        "qwen2",
        base_url=os.environ["OPENAI_API_BASE"],
        api_key=os.environ["OPENAI_API_KEY"],
        timeout=60.0 
    )
    return llm, embeddings

In [5]:
import neo4j
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
driver        = neo4j.GraphDatabase.driver(NEO4J_URI,
                                              auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
                                              database=NEO4J_DATABASE)
llm, embedder = make_clients()
splitter      = FixedSizeSplitter(400, 40)

In [6]:
NODE_TYPES = [
    "Bill", "Title", "Subtitle", "Section", "Agency", "Program",
    "Requirement", "Appropriation", "BeneficiaryGroup",
    "Deadline", "Act"
]

RELATIONSHIP_TYPES = [
    "HAS_TITLE", "HAS_SUBTITLE", "HAS_SECTION", "ESTABLISHES",
    "AUTHORIZES_FUNDS", "APPROPRIATES_TO", "IMPOSES_REQUIREMENT",
    "BENEFITS", "AMENDS", "ADMINISTERED_BY", "AFFECTS",
    "APPLIES_TO", "EFFECTIVE_ON"
]

PATTERNS = [
    ("Bill", "HAS_TITLE", "Title"),
    ("Title", "HAS_SUBTITLE", "Subtitle"),
    ("Subtitle", "HAS_SECTION", "Section"),
    ("Section", "HAS_SECTION", "Section"),          # nested §§
    ("Section", "ESTABLISHES", "Program"),
    ("Section", "AUTHORIZES_FUNDS", "Appropriation"),
    ("Section", "IMPOSES_REQUIREMENT", "Requirement"),
    ("Section", "BENEFITS", "BeneficiaryGroup"),
    ("Section", "AMENDS", "Act"),
    ("Program", "ADMINISTERED_BY", "Agency"),
    ("Program", "BENEFITS", "BeneficiaryGroup"),
    ("Appropriation", "APPROPRIATES_TO", "Program"),
    ("Requirement", "AFFECTS", "Agency"),
    ("Requirement", "APPLIES_TO", "BeneficiaryGroup"),
    ("Section", "EFFECTIVE_ON", "Deadline"),
]

In [7]:
def build_schema_section() -> str:
    """Yield a markdown bulleted list that the LLM will see."""
    nodes = "\n".join(f"* **{n}**" for n in NODE_TYPES)
    rels  = "\n".join(f"* `{r}`"  for r in RELATIONSHIP_TYPES)
    return f"""
**Node labels**

{nodes}

**Relationship types**

{rels}
""".strip()

In [8]:
prompt_template = """
You are a legislative analyst tasked with extracting structured information from U.S. federal bills and representing it as a property graph that will power a GraphRAG question-answering system.

**Task**

1. **Extract every entity** (node) the *Input text* clearly mentions and assign
   it one of the allowed labels below.
2. **Extract every relationship** that exists *within* the *Input text*, using
   the correct direction (from the `start_node_id` to the `end_node_id`).

**Output format**

Return **only** a single JSON object in the form

{{{{
  "nodes": [
    {{{{ "id": "0", "label": "<NodeLabel>", "properties": {{{{ "name": "<entity name>" }}}} }}}},
    ...
  ],
  "relationships": [
    {{{{ "type": "<REL_TYPE>", "start_node_id": "0", "end_node_id": "1", "properties": {{{{ "details": "<optional short description>" }}}} }}}},
    ...
  ]
}}}}

* Each `id` must be unique *within this chunk*.
* Include only properties that appear literally in the text.
* If the input is empty return `{{}}`.

**Allowed node labels and relationship types**

{schema}

**Guidelines**

* Do **not** invent information not present in the text.
* Use the containment edges (`HAS_TITLE`, `HAS_SUBTITLE`, `HAS_SECTION`).
* Use policy-specific edges (`APPROPRIATES_TO`, `EFFECTIVE_ON`, …) only when
  the text supports them.
* Keep entity types general so they can be reused across bills.

**Few-shot examples**

{examples}

---

**Input text**

{text}
"""

In [9]:
prompt_ready = prompt_template.format(
    schema=build_schema_section(),
    examples="",
    text="{text}"
)

In [10]:
print(prompt_ready)


You are a legislative analyst tasked with extracting structured information from U.S. federal bills and representing it as a property graph that will power a GraphRAG question-answering system.

**Task**

1. **Extract every entity** (node) the *Input text* clearly mentions and assign
   it one of the allowed labels below.
2. **Extract every relationship** that exists *within* the *Input text*, using
   the correct direction (from the `start_node_id` to the `end_node_id`).

**Output format**

Return **only** a single JSON object in the form

{{
  "nodes": [
    {{ "id": "0",
       "label": "<NodeLabel>",
       "properties": {{ "name": "<entity name>" }} }},
    …
  ],
  "relationships": [
    {{ "type": "<REL_TYPE>",
       "start_node_id": "0",
       "end_node_id": "1",
       "properties": {{ "details": "<optional short description>" }} }},
    …
  ]
}}

* Each `id` must be unique *within this chunk*.
* Include only properties that appear literally in the text.
* If the input is e

In [11]:
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=driver,
    text_splitter=FixedSizeSplitter(chunk_size=500, chunk_overlap=100),
    embedder=embedder,
    entities=NODE_TYPES,
    relations=RELATIONSHIP_TYPES,
    enforce_schema="STRICT",
    prompt_template = prompt_template.format(
        schema   = build_schema_section(),
        examples = None,
        text     = "{text}"
    ),
    from_pdf=True
)

In [12]:
pdf_file_paths = ['BILLS-119hr1rh-sample.pdf']

for path in pdf_file_paths:
    print(f"Processing : {path}")
    pdf_result = await kg_builder.run_async(file_path=path)
    print(f"Result: {pdf_result}")


Processing : BILLS-119hr1rh-sample.pdf


IndexError: Replacement index 0 out of range for positional args tuple

In [ ]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, name="text_embeddings", label="Chunk",
                    embedding_property="embedding", dimensions=1024, similarity_fn="cosine")

# STOP

## Knowledge Graph Retrieval

In [ ]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

In [ ]:
# … rest of your notebook stays unchanged …
chunks  = splitter(text)
vectors = embedder.embed([c.text for c in chunks])
result  = await llm.ainvoke(prompt_template.format(text=text))

In [ ]:
prompt = prompt_template.format(text=some_text, schema=my_schema_json)
result = await llm.ainvoke(prompt)

In [ ]:
llm, embedder = make_clients()

chunks  = splitter.split(text)
vectors = embedder.embed([c.text for c in chunks])
result  = await llm.ainvoke(prompt)

In [ ]:
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder_pdf = SimpleKGPipeline(
    llm=ex_llm,
    driver=driver,
    text_splitter=FixedSizeSplitter(chunk_size=500, chunk_overlap=100),
    embedder=embedder,
    entities=node_labels,
    relations=rel_types,
    prompt_template=prompt_template,
    from_pdf=True
)

Below, we run the `SimpleKGPipeline` to construct our knowledge graph from 3 pdf documents and store in Neo4j.

In [ ]:
pdf_file_paths = ['BILLS-119hr1rh.pdf']

for path in pdf_file_paths:
    print(f"Processing : {path}")
    pdf_result = await kg_builder_pdf.run_async(file_path=path)
    print(f"Result: {pdf_result}")

## Knowledge Graph Retrieval

We will leverage Neo4j's vector search capabilities here. To do this, we need to begin by creating a vector index on the text chunks from the PDFs, which are stored on `Chunk` nodes in our knowledge graph.

In [ ]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, name="text_embeddings", label="Chunk",
                    embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

Now that the index is set up, we will start simple with a __VectorRetriever__.  The __VectorRetriever__ just queries `Chunk` nodes via vector search, bringing back the text and some metadata.

In [ ]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

Below we visualize the context we get back when submitting a search prompt. 

In [ ]:
import json

vector_res = vector_retriever.get_search_results(query_text = "How is precision medicine applied to Lupus?", 
                                                 top_k=3)
for i in vector_res.records: print("====\n" + json.dumps(i.data(), indent=4))

The GraphRAG Python Package offers [a wide range of useful retrievers](https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#retriever-configuration), each covering different knowledge graph retrieval patterns.

Below we will use the __`VectorCypherRetriever`__, which allows you to run a graph traversal after finding nodes with vector search.  This uses Cypher, Neo4j's graph query language, to define the logic for traversing the graph. 

As a simple starting point, we'll traverse up to 3 hops out from each Chunk, capture the relationships encountered, and include them in the response alongside our text chunks.


In [ ]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

vc_retriever = VectorCypherRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks, 
  collect(DISTINCT rel) AS rels

//3) format and return context
RETURN '=== text ===\n' + apoc.text.join([c in chunks | c.text], '\n---\n') + '\n\n=== kg_rels ===\n' +
  apoc.text.join([r in rels | startNode(r).name + ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' + endNode(r).name ], '\n---\n') AS info
"""
)

Below we visualize the context we get back when submitting a search prompt. 

In [ ]:
vc_res = vc_retriever.get_search_results(query_text = "How is precision medicine applied to Lupus?", top_k=3)

# print output
kg_rel_pos = vc_res.records[0]['info'].find('\n\n=== kg_rels ===\n')
print("# Text Chunk Context:")
print(vc_res.records[0]['info'][:kg_rel_pos])
print("# KG Context From Relationships:")
print(vc_res.records[0]['info'][kg_rel_pos:])

## GraphRAG
 
 You can construct GraphRAG pipelines with the `GraphRAG` class.  At a minimum, you will need to pass the constructor an LLM and a retriever. You can optionally pass a custom prompt template. We will do so here just to provide a bit more guidance for the LLM to stick to information from our data source.
 
Below we create `GraphRAG` objects for both the vector and vector-cypher retrievers. 

In [ ]:
from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.generation import RagTemplate
from neo4j_graphrag.generation.graphrag import GraphRAG

llm = LLM(model_name="qwen2",  model_params={"temperature": 0.0})

rag_template = RagTemplate(template='''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned. 

# Question:
{query_text}
 
# Context:
{context}

# Answer:
''', expected_inputs=['query_text', 'context'])

v_rag  = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)
vc_rag = GraphRAG(llm=llm, retriever=vc_retriever, prompt_template=rag_template)

Now we can run GraphRAG and examine the outputs. 

In [ ]:
q = "How is precision medicine applied to Lupus? provide in list format."
print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k':5}).answer}")
print("\n===========================\n")
print(f"Vector + Cypher Response: \n{vc_rag.search(q, retriever_config={'top_k':5}).answer}")

In [ ]:
q = "Can you summarize systemic lupus erythematosus (SLE)? including common effects, biomarkers, and treatments? Provide in detailed list format."

v_rag_result = v_rag.search(q, retriever_config={'top_k': 5}, return_context=True)
vc_rag_result = vc_rag.search(q, retriever_config={'top_k': 5}, return_context=True)

print(f"Vector Response: \n{v_rag_result.answer}")
print("\n===========================\n")
print(f"Vector + Cypher Response: \n{vc_rag_result.answer}")

In [ ]:
for i in v_rag_result.retriever_result.items: print(json.dumps(eval(i.content), indent=1))

In [ ]:
vc_ls = vc_rag_result.retriever_result.items[0].content.split('\\n---\\n')
for i in vc_ls:
    if "biomarker" in i: print(i)

In [ ]:
vc_ls = vc_rag_result.retriever_result.items[0].content.split('\\n---\\n')
for i in vc_ls:
    if "treat" in i: print(i)

In [ ]:
q = "Can you summarize systemic lupus erythematosus (SLE)? including common effects, biomarkers, treatments, and current challenges faced by Physicians and patients? provide in list format with details for each item."
print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k': 5}).answer}")
print("\n===========================\n")
print(f"Vector + Cypher Response: \n{vc_rag.search(q, retriever_config={'top_k': 5}).answer}")